# **DB 생성 및 관리**

## **1.환경준비**

In [1]:
import pandas as pd
import os
import sqlite3

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## **2.DB 작업**

### (1) DB 생성 및 연결

* 데이터베이스 연결
    * 폴더는 미리 생성
    * 해당 파일이 없으면 새로 생성, 있으면 연결

In [2]:
path = 'testdb.db'
conn = sqlite3.connect(path)

### (2) DB 작업 절차

* 1) 커서 활용 : 대부분의 SQL 작업 가능
    * ① 연결
    * ② 커서 선언
    * ③ 작업 : sql
    * ④ 커밋(수정작업 저장)
    * ⑤ 연결종료

* 2) pandas 함수 활용 : 주로 insert, update, delete, select, create table & insert
    * ① 연결
    * ② 작업 : pandas 함수 이용
    * ③ 연결종료

#### 1) 테이블 생성
* 테이블 생성 작업
    * DB Browser for SQLite 에서 직접 수행
    * 파이썬에서 **연결**하고, **커서**를 생성해서 작업

In [3]:
# 커서 객체 생성
cursor = conn.cursor()

# test 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS test (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER NOT NULL
)
''')

# 변경사항 커밋 (저장)
conn.commit()

# 연결 종료
conn.close()

#### 2) insert
* 데이터프레임에서 직접 DB로 보내기 : df.to_sql

In [4]:
# ① 연결
conn = sqlite3.connect(path)

# ② 작업 : to_sql
data = pd.DataFrame({'name': ['Alice', 'Bob', 'Charlie', 'David'], 'age': [30, 25, 35, 40]})
data.to_sql('test', conn, if_exists='append', index=False) # test 테이블이 있으면 insert, 없으면 생성

# ③ 연결 종료
conn.close()

#### 3) Select

In [5]:
# ① 연결
conn = sqlite3.connect(path)

# ② 작업 : select
df = pd.read_sql('SELECT * FROM test', conn)
display(df)

# ③ 연결 종료
conn.close()

,id,name,age
0,1,Alice,30
1,2,Bob,25
2,3,Charlie,35
3,4,David,40


#### 4) Drop

* 커서로만 작업 가능

In [ ]:
# ① 연결
conn = sqlite3.connect(path)

# ② 커서 선언
cursor = conn.cursor()

# ③ 작업 : 테이블 삭제
cursor.execute('DROP TABLE IF EXISTS test')

# ④ 커밋(수정작업 저장)
conn.commit()

# ⑤ 연결종료
conn.close()

## **3.관리용 DB**


### (1) DB 생성

* DB를 아래 경로와 이름으로 생성
    * project7/db/em.db

In [5]:
# 데이터베이스 경로 설정
db_dir = "db"  # 폴더 경로
db_name = "em.db"       # 데이터베이스 파일 이름
db_path = os.path.join(db_dir, db_name)

conn = sqlite3.connect(db_path)

### (2) log 테이블 생성
* 칼럼 구성

|column name | descrition | data type | comment |
|----|----|----|----|
|datetime | 요청일시 | text | yyyy-mm-dd hh:mi:ss |
|input_text | 입력text | text |  |
|input_latitude | 입력 위도 | real |  |
|input_longitude | 입력 경도 | real |  |
|em_class | 응급 등급(1~5) | integer |  |
|hospital1 | 추천병원명 | text |  |
|addr1 | 추천병원주소 | text |  |
|tel1 | 추천병원전화번호 | text |  |
|hospital2 | 추천병원명2 | text |  |
|addr2 | 추천병원주소2 | text |  |
|tel2 | 추천병원전화번호2 | text |  |
|hospital3 | 추천병원명3 | text |  |
|addr3 | 추천병원주소3 | text |  |
|tel3 | 추천병원전화번호3 | text |  |

In [6]:
# 커서 객체 생성
cursor = conn.cursor()

# log 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS log (
    datetime TEXT,                -- 요청일시 (yyyy-mm-dd hh:mi:ss)
    input_text TEXT,              -- 입력 텍스트
    input_latitude REAL,          -- 입력 위도
    input_longitude REAL,         -- 입력 경도
    em_class INTEGER,             -- 응급 등급 (1~5)
    hospital1 TEXT,               -- 추천 병원명1
    addr1 TEXT,                   -- 추천 병원주소1
    tel1 TEXT,                    -- 추천 병원전화번호1
    hospital2 TEXT,               -- 추천 병원명2
    addr2 TEXT,                   -- 추천 병원주소2
    tel2 TEXT,                    -- 추천 병원전화번호2
    hospital3 TEXT,               -- 추천 병원명3
    addr3 TEXT,                   -- 추천 병원주소3
    tel3 TEXT                     -- 추천 병원전화번호3
)
''')

# 변경사항 커밋 (저장)
conn.commit()

# 연결 종료
conn.close()

In [9]:
# df 확인

# ① 연결
conn = sqlite3.connect(db_path)

# ② 작업 : select
df = pd.read_sql('SELECT * FROM log', conn)
display(df)

# ③ 연결 종료
conn.close()

,datetime,input_text,input_latitude,input_longitude,em_class,hospital1,addr1,tel1,hospital2,addr2,tel2,hospital3,addr3,tel3


### (3) log 테이블 입력 함수

* 현재 일시 저장하기 예제
            # 현재 연월일 시분초 뽑기
            from datetime import datetime
            dt = datetime.now()
            dt = dt.strftime('%Y-%m-%d %H:%M:%S')


In [8]:
from datetime import datetime

def insert_log(input_text, input_latitude, input_longitude, em_class,
               hospital1, addr1, tel1, hospital2, addr2, tel2, hospital3, addr3, tel3):

    dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    db_path = "project7/db/em.db"
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute('''
        INSERT INTO log (
            datetime, input_text, input_latitude, input_longitude, em_class,
            hospital1, addr1, tel1, hospital2, addr2, tel2, hospital3, addr3, tel3
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (dt, input_text, input_latitude, input_longitude, em_class,
          hospital1, addr1, tel1, hospital2, addr2, tel2, hospital3, addr3, tel3))

    conn.commit()
    conn.close()

    print("데이터가 성공적으로 입력되었습니다.")

In [ ]:
# 1. 준비 작업 ---------------------------------------------
openai.api_key = em.load_file(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

map_key = em.load_file(path + 'map_key.txt')
map_key = json.loads(map_key)
c_id, c_key = 'hqev5yhpp0', '0hKV6LdcBYambAIlHkVtAMuxCuRD6ypCVQrSGjne'

emergency = pd.read_csv(path + '응급실 정보.csv')

# 모델, 토크나이저 로드
save_directory = path + "fine_tuned_bert"
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)

# 2. 데이터 처리(pipeline) ---------------------------------
# input
audio_path = path + 'audio/'
filename = 'audio1.mp3'

# 처리: 음성 -> 텍스트 -> 요약
result = em.audio2text(audio_path, filename)
result = em.text2summary(result)

# 응급 등급 예측
predicted_class, _ = em.predict(result, model, tokenizer)

# 테스트 input --------------------
predicted_class = 2  # 테스트용
start_lat, start_lng = 35.548238, 129.307011
# ---------------------------------

# 병원 추천 및 결과 처리
emergency_list = emergency.to_dict('records')
if predicted_class <= 2:
    hospital_results = em.recommend_hospital3(emergency_list, start_lat, start_lng, c_id, c_key)

    # 결과를 DB에 삽입
    insert_hospital_data(
        input_text=result,
        input_latitude=start_lat,
        input_longitude=start_lng,
        em_class=predicted_class,
        hospitals=hospital_results
    )
    print("병원 추천 결과가 데이터베이스에 저장되었습니다.")

else:
    print('개인 건강관리')